# Solvers ⚙️

In this exercise, you will investigate the effects of different `solvers` on `LogisticRegression` models.

👇 Run the code below to import the dataset

In [2]:
import pandas as pd

df = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers_dataset.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol,quality rating
0,9.47,5.97,7.36,10.17,6.84,9.15,9.78,9.52,10.34,8.80,6
1,10.05,8.84,9.76,8.38,10.15,6.91,9.70,9.01,9.23,8.80,7
2,10.59,10.71,10.84,10.97,9.03,10.42,11.46,11.25,11.34,9.06,4
3,11.00,8.44,8.32,9.65,7.87,10.92,6.97,11.07,10.66,8.89,8
4,12.12,13.44,10.35,9.95,11.09,9.38,10.22,9.04,7.68,11.38,3


- The dataset consists of different wines 🍷
- The features describe different properties of the wines 
- The target 🎯 is a quality rating given by an expert

## 1. Target engineering

In this section, you are going to transform the ratings into a binary target.

👇 How many observations are there for each rating?

In [3]:
df['quality rating'].value_counts()

10    10143
5     10124
1     10090
2     10030
8      9977
6      9961
9      9955
7      9954
4      9928
3      9838
Name: quality rating, dtype: int64

In [4]:
df['quality rating'].unique()

array([ 6,  7,  4,  8,  3,  1,  2, 10,  5,  9])

❓ Create `y` by transforming the target into a binary classification task where quality ratings below 6 are bad [0], and ratings of 6 and above are good [1]

In [5]:
def change_rating(rating):
    if rating < 6:
        return 0
    else:
        return 1

In [6]:
y = df['quality rating'].apply(change_rating)

In [7]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol,quality rating
0,9.47,5.97,7.36,10.17,6.84,9.15,9.78,9.52,10.34,8.80,6
1,10.05,8.84,9.76,8.38,10.15,6.91,9.70,9.01,9.23,8.80,7
2,10.59,10.71,10.84,10.97,9.03,10.42,11.46,11.25,11.34,9.06,4
3,11.00,8.44,8.32,9.65,7.87,10.92,6.97,11.07,10.66,8.89,8
4,12.12,13.44,10.35,9.95,11.09,9.38,10.22,9.04,7.68,11.38,3


❓ Check the class balance of the new binary target

In [8]:
y.value_counts()

0    50010
1    49990
Name: quality rating, dtype: int64

❓ Create your `X` by normalising the features. This will allow for fair comparison of different solvers.

In [9]:
X = df.drop(columns=['quality rating'])

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
mm_scaler = MinMaxScaler().fit(X)

In [12]:
X_scaled = mm_scaler.transform(X)

In [13]:
X_scaled = pd.DataFrame(X_scaled)
X_scaled.columns = X.columns

## 2. LogisticRegression solvers

❓ Logistic Regression models can be optimized using different **solvers**. Make a comparison of the available solvers':
- Fit time - which solver is **the fastest**?
- Precision - **how different** are their respective precision scores?

Available solvers for Logistic Regression are `['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']`
 
For more information on these 5 solvers, check out [this Stack Overflow thread](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions)

In [14]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

scores = []
fit_times = []

for solver in solvers:
    cv_log = cross_validate(
        LogisticRegression(solver=solver),
        X_scaled,
        y,
        cv=5,
        scoring = ['precision']
    )
    
    scores.append(cv_log['test_precision'].mean())
    fit_times.append(cv_log['fit_time'].mean())

In [16]:
solvers_performance = pd.DataFrame({"Precision score": scores, "Fit times": fit_times}, index=solvers)
solvers_performance

,Precision score,Fit times
newton-cg,0.874386,0.227197
lbfgs,0.874389,0.293124
liblinear,0.874449,0.210093
sag,0.874371,0.548363
saga,0.874386,0.791048


In [17]:
cv_log

{'fit_time': array([0.83490825, 0.76140499, 0.785151  , 0.75415874, 0.81961799]),
 'score_time': array([0.00987601, 0.01000333, 0.0100131 , 0.01012707, 0.01007795]),
 'test_precision': array([0.87637162, 0.87376828, 0.87458472, 0.87488319, 0.87232298])}

In [18]:
# YOUR ANSWER
fastest_solver = solvers_performance['Fit times'].idxmin()
fastest_solver

'liblinear'

<details>
    <summary>ℹ️ Click here for our interpretation</summary>

All solvers should produce similar precision scores because our cost-function is "easy" enough to have a global minimum which is found by all 5 solvers. For very complex cost-functions such as in Deep Learning, different solvers may stopping at different values of the loss function.

**The wine dataset**
    
If you check feature importance with sklearn's <a href="https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html">permutation_importance</a> on the current dataset, you'll see many features result in almost 0 importance. Liblinear solver successively moves only along *one* direction at a time, regularizing the others with L1 regularization (a.k.a, setting their beta to 0), which might provide a good fit for a dataset where many features are not that important in predicting the target.

</details> 

###  🧪 Test your code

In [19]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'solvers',
    fastest_solver=fastest_solver
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/bingobango/.pyenv/versions/tom/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bingobango/code/lewagon/data-solvers/tests
plugins: anyio-3.6.1, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 1 item

test_solvers.py::TestSolvers::test_fastest_solver PASSED                 [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/solvers.pickle

git commit -m 'Completed solvers step'

git push origin master



## 3. Stochastic Gradient Descent

Logistic Regression models can also be optimized via Stochastic Gradient Descent.

❓ Evaluate a Logistic Regression model optimized via **Stochastic Gradient Descent**. How do its precision score and training time compare to the performance of the models trained in section 2?


<details>
<summary>💡 Hint</summary>

- If you are stuck, look at the [SGDClassifier doc](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)!

</details>



In [20]:
from sklearn.linear_model import SGDClassifier

In [21]:
sgd_model = SGDClassifier(loss='log_loss')


In [22]:
cv_sgd = cross_validate(
    sgd_model,
    X_scaled,
    y,
    cv=5,
    scoring=['precision']
)

In [23]:
cv_sgd

{'fit_time': array([0.14856219, 0.14938498, 0.13996816, 0.14874911, 0.14273   ]),
 'score_time': array([0.01080871, 0.01010203, 0.00992775, 0.01007175, 0.01048613]),
 'test_precision': array([0.8777288 , 0.88532649, 0.86103296, 0.8830962 , 0.87132353])}

In [24]:
print(f"test score: {cv_sgd['test_precision'].mean()}")
print(f"time score: {cv_sgd['fit_time'].mean()}")

test score: 0.8757015958483972
time score: 0.14587888717651368


In [ ]:
# SGD only uses 1 sample or mini batch per step

☝️ The SGD model should have one of the shortest times (maybe even shorter than `liblinear`), for similar performance. This is a direct effect of performing each epoch of the Gradient Descent on a single row as opposed to loading 100k rows into memory at a time.

## 4. Predictions

❓ Use the best model (balanced with short fit time and high precision) to predict the binary quality (0 or 1) of the following wine. Store your:
- `predicted_class`
- `predicted_proba_of_class`

In [25]:
new_wine = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers_new_wine.csv')
new_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol
0,9.54,13.5,12.35,8.78,14.72,9.06,9.67,10.15,11.17,12.17


In [41]:
X_scaled

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol
0,0.531348,0.285244,0.265966,0.504968,0.229879,0.363248,0.451878,0.432173,0.557503,0.413523
1,0.576803,0.420113,0.459984,0.343270,0.412348,0.123932,0.442488,0.370948,0.435926,0.413523
2,0.619122,0.507989,0.547292,0.577236,0.350606,0.498932,0.649061,0.639856,0.667032,0.432028
3,0.651254,0.401316,0.343573,0.457995,0.286659,0.552350,0.122066,0.618247,0.592552,0.419929
4,0.739028,0.636278,0.507680,0.485095,0.464168,0.387821,0.503521,0.374550,0.266156,0.597153
...,...,...,...,...,...,...,...,...,...,...
99995,0.332288,0.215695,0.337914,0.363144,0.371555,0.568376,0.287559,0.596639,0.785323,0.427046
99996,0.617555,0.453947,0.465643,0.397471,0.391400,0.458333,0.579812,0.643457,0.473165,0.425623
99997,0.590909,0.520677,0.620049,0.648600,0.341786,0.350427,0.469484,0.472989,0.524644,0.403559
99998,0.357367,0.190320,0.239289,0.390244,0.320838,0.427350,0.647887,0.515006,0.336254,0.459075


In [28]:
# train model
best_model = sgd_model.fit(X_scaled, y)

In [40]:
# scale new x features using original scaler
# dont fit on new data
new_x = pd.DataFrame(mm_scaler.transform(new_wine))
new_x.columns = new_wine.columns

In [32]:
new_x

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol
0,0.536834,0.639098,0.669361,0.379404,0.664278,0.353632,0.438967,0.507803,0.648412,0.653381


In [33]:
best_model.predict(new_x)

array([0])

In [37]:
predicted_class = best_model.predict(new_x)[0]
predicted_class

0

In [38]:
predicted_proba_of_class = best_model.predict_proba(new_x)[0][0]
predicted_proba_of_class

0.9623004182871935

# 🏁  Check your code and push your notebook

In [39]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'new_data_prediction',
    predicted_class=predicted_class,
    predicted_proba_of_class=predicted_proba_of_class
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/bingobango/.pyenv/versions/tom/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bingobango/code/lewagon/data-solvers/tests
plugins: anyio-3.6.1, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 2 items

test_new_data_prediction.py::TestNewDataPrediction::test_predicted_class PASSED [ 50%]
test_new_data_prediction.py::TestNewDataPrediction::test_predicted_proba PASSED [100%]

============================== 2 passed in 0.17s ===============================


💯 You can commit your code:

git add tests/new_data_prediction.pickle

git commit -m 'Completed new_data_prediction step'

git push origin master

